In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.optimizers import Adam


# LSTM implementation

In [17]:
rand = 0xC0FFEE

# Load the dataset
df = pd.read_csv('combined_data.csv')

# Handle missing values if any
df.fillna(method='ffill', inplace=True)

# Encode the target variable
label_encoder = LabelEncoder()
df['activity'] = label_encoder.fit_transform(df['activity'])

# Scale the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df.drop(columns=['date_time', 'activity']))

# Prepare the data for LSTM
X = []
y = []

# Define the number of time steps
time_steps = 10

for i in range(time_steps, len(scaled_features)):
    X.append(scaled_features[i-time_steps:i])
    y.append(df['activity'].iloc[i])

X, y = np.array(X), np.array(y)

# One-hot encode the target variable
y = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=rand, stratify=y)

In [18]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
508/508 [==============================] - 4s 4ms/step - loss: 0.0743 - accuracy: 0.9776 - val_loss: 0.0148 - val_accuracy: 0.9963
Epoch 2/50
508/508 [==============================] - 2s 4ms/step - loss: 0.0163 - accuracy: 0.9950 - val_loss: 0.0117 - val_accuracy: 0.9969
Epoch 3/50
508/508 [==============================] - 2s 4ms/step - loss: 0.0139 - accuracy: 0.9957 - val_loss: 0.0122 - val_accuracy: 0.9967
Epoch 4/50
508/508 [==============================] - 2s 4ms/step - loss: 0.0105 - accuracy: 0.9964 - val_loss: 0.0074 - val_accuracy: 0.9983
Epoch 5/50
508/508 [==============================] - 2s 4ms/step - loss: 0.0096 - accuracy: 0.9968 - val_loss: 0.0077 - val_accuracy: 0.9974
Epoch 6/50
508/508 [==============================] - 2s 4ms/step - loss: 0.0069 - accuracy: 0.9975 - val_loss: 0.0051 - val_accuracy: 0.9987
Epoch 7/50
508/508 [==============================] - 2s 4ms/step - loss: 0.0062 - accuracy: 0.9977 - val_loss: 0.0054 - val_accuracy: 0.9987
Epoch 

In [20]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

170/170 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 0.9996
Test Accuracy: 99.96%


In [21]:
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Decode the predicted and true classes
predicted_labels = label_encoder.inverse_transform(predicted_classes)
true_labels = label_encoder.inverse_transform(true_classes)

# Print some predictions
for i in range(10):
    print(f'Predicted: {predicted_labels[i]}, True: {true_labels[i]}')

170/170 [==============================] - 1s 1ms/step
Predicted: walking, True: walking
Predicted: cycling, True: cycling
Predicted: walking, True: walking
Predicted: walking, True: walking
Predicted: cycling, True: cycling
Predicted: cycling, True: cycling
Predicted: walking, True: walking
Predicted: cycling, True: cycling
Predicted: walking, True: walking
Predicted: cycling, True: cycling
